Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Studying the feature vectors in the feature space

In [1]:
import os
from six.moves import xrange  
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage import io
import warnings
import scipy
warnings.filterwarnings('ignore')

# Custom scripts
import lscript as lsp
import modelling as md

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
crop_size=112

cluster_length=16
feature_size=8192 
nb_classes=2
saved_path='/nobackup/leopauly/S2l/MIME/90_10_shuffle/'

batch_size=32
memory_batch_size_train=266
memory_batch_size_test=170
next_batch_start=0
sample_batch_size=10

####  Loading model

In [3]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 
y_true = tf.placeholder(tf.float32, [None, nb_classes],name='y_true')
y_true_cls = tf.placeholder(tf.int64, [None],name='y_true_cls')

model_keras = md.C3D_MIME20_training_model_tf(summary=True)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)
y_pred_cls = tf.argmax(out, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

####  Feature extraction and Distance calculation

In [4]:
## Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-155'))
print("Model restored from file: %s" % saved_path,flush=True)

INFO:tensorflow:Restoring parameters from /nobackup/leopauly/S2l/MIME/90_10_shuffle/activity_model.ckpt-155
Model restored from file: /nobackup/leopauly/S2l/MIME/90_10_shuffle/


In [5]:
## Uniform sampling of frames
def get_compress_frames_data(filename, num_frames_per_clip=16):
  ''' Given a directory containing extracted frames, return a video clip of
  (num_frames_per_clip) consecutive frames as a list of np arrays '''
  ret_arr = []
  for parent, dirnames, filenames in os.walk(filename):

    filenames = sorted(filenames)
    jump=math.floor((len(filenames)/num_frames_per_clip))
    loop=0
    for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        image_name = str(filename) + '/' + str(filenames[i])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr) #ret_arr=ret_arr/255
  return np.array(ret_arr)

In [6]:
norm_order=.05
def dist_calc(value_a,value_b):
    #distance_=np.linalg.norm(value_a-value_b,ord=norm_order)
    #distance_=scipy.spatial.distance.cosine(value_a,value_b)
    distance_=sklearn.metrics.pairwise.cosine_similarity(value_a,value_b)
    return(distance_)

In [7]:
## Extraction of features 
def extract_video_features(vid):
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name('flatten_1/Reshape:0') 
    #'flatten_1/Reshape:0' #dropout_1/cond/Merge:0 #fc8/BiasAdd:0 #Softmax:0
    f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 })
    features=np.reshape(f_v_val,(-1))
    return features

------------------------------------------------------------------------------------------------------------------

#### Reach

In [8]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            if(sub_dir_a_!=sub_dir_b_):
                #print(sub_dir_b_)
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('\nInter class variance bw class :',intra_var_class)
print('Number of combinations:',np.shape(dist_sim_class_nozero))

intra_reach=intra_var_class

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_bg', 'reach_robo_fast', 'reach_robo_obj2']

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_bg', 'reach_robo_fast', 'reach_robo_obj2']

Inter class variance bw class : [[ 0.71008414]]
Number of combinations: (56, 1, 1)


In [9]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter1_reach=inter_var

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_bg', 'reach_robo_fast', 'reach_robo_obj2']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_bg_fast', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new']

Inter class variance bw class 1 and class 2: [[ 0.68178189]]
Number of combinations: (64, 1, 1)


In [10]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach_n_Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter2_reach=inter_var

Sub directories: ['multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_bg', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2']

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_bg', 'reach_robo_fast', 'reach_robo_obj2']

Inter class variance bw class 1 and class 2: [[ 0.64337826]]
Number of combinations: (64, 1, 1)


------------------------------------------------------------------------------------------------------------------

#### Push

In [11]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter1_push=inter_var

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_bg', 'reach_robo_fast', 'reach_robo_obj2']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_bg_fast', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new']

Inter class variance bw class 1 and class 2: [[ 0.68178189]]
Number of combinations: (64, 1, 1)


In [12]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            if(sub_dir_a_!=sub_dir_b_):
                #print(sub_dir_b_)
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('\nInter class variance bw class :',intra_var_class)
print('Number of combinations:',np.shape(dist_sim_class_nozero))

intra_push=intra_var_class

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_bg_fast', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_bg_fast', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new']

Inter class variance bw class : [[ 0.73042142]]
Number of combinations: (56, 1, 1)


In [13]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach_n_Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter2_push=inter_var

Sub directories: ['multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_bg', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_bg_fast', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new']

Inter class variance bw class 1 and class 2: [[ 0.58378214]]
Number of combinations: (64, 1, 1)


------------------------------------------------------------------------------------------------------------------

#### Push n Reach 

In [14]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach_n_Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

inter1_rnp=inter_var

['.DS_Store', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_bg', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2']
['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_bg', 'reach_robo_fast', 'reach_robo_obj2']
Inter class variance bw class 1 and class 2: [[ 0.64337826]]
Shape (64, 1, 1)


In [15]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach_n_Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

inter2_rnp=inter_var

['.DS_Store', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_bg', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2']
['.DS_Store', 'push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_bg_fast', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new']
Inter class variance bw class 1 and class 2: [[ 0.58378214]]
Shape (64, 1, 1)


In [16]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach_n_Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Exp.A.1._Easytasks/Reach_n_Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            if(sub_dir_a_!=sub_dir_b_):
                #print(sub_dir_b_)
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=dist_calc(extract_video_features(vid_a),extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('Inter class variance bw class :',intra_var_class)


intra_rnp=intra_var_class

['.DS_Store', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_bg', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2']
['.DS_Store', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_bg', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2']
Inter class variance bw class : [[ 0.70045668]]


------------------------------------------------------------------------------------------------------------------

In [17]:
##Ratios
ratio1_reach=round(intra_reach/intra_reach,2)
ratio2_reach=round(intra_reach/inter1_reach,2)
ratio3_reach=round(intra_reach/inter2_reach,2)

print('---',ratio1_reach)
print(ratio2_reach)
print(ratio3_reach)

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
##Ratios
ratio1_push=round(intra_push/inter1_push,2)
ratio2_push=round(intra_push/intra_push,2)
ratio3_push=round(intra_push/inter2_push,2)

print(ratio1_push)
print('---',ratio2_push)
print(ratio3_push)

In [ ]:
##Ratios
ratio1_rnp=round(intra_rnp/inter1_rnp,2)
ratio2_rnp=round(intra_rnp/inter2_rnp,2)
ratio3_rnp=round(intra_rnp/intra_rnp,2)

print(ratio1_rnp)
print(ratio2_rnp)
print('---',ratio3_rnp)

In [ ]:
class_overlap_ratio=round(((ratio2_reach+ratio3_reach+ratio1_push+ratio3_push+ratio1_rnp+ratio2_rnp)/6),2)
print(class_overlap_ratio)